In [1]:
import tensorflow as tf
import tensorlayer as tl
from tensorlayer.layers import DenseLayer, EmbeddingInputlayer, retrieve_seq_length_op2
from tensorlayer.models.seq2seq import Seq2seq 
from nltk.tokenize import  word_tokenize
from sklearn.utils import shuffle
from tqdm import tqdm
import numpy as np
import pickle 
# from data.data import load_data

In [2]:
path = './data/'

In [3]:
def load_data(path):
	# read data control dictionaries
	try:
		with open(path + 'metadata.pkl', 'rb') as f:
			metadata = pickle.load(f)
	except:
		metadata = None
	# read numpy arrays
	idx_q = np.load(path + 'idx_q.npy', encoding = 'latin1')
	idx_a = np.load(path + 'idx_a.npy', encoding = 'latin1')
	return metadata, idx_q, idx_a

In [4]:
with open('./data/metadata.pkl', 'rb') as f:
	metadata = pickle.load(f)

In [5]:
print(metadata)

{'w2idx': {'_': 0, 'unk': 1, 'the': 2, 'a': 3, 'is': 4, 'of': 5, 'algorithm': 6, 'to': 7, 'in': 8, 'what': 9, 'you': 10, 'do': 11, 'and': 12, 'by': 13, 'that': 14, 'an': 15, 'for': 16, 'are': 17, 'how': 18, 'define': 19, 'mean': 20, 'explain': 21, 'it': 22, 'or': 23, 'process': 24, 'tree': 25, 'search': 26, 'system': 27, 'time': 28, 'sort': 29, 'as': 30, 'which': 31, 'graph': 32, 'data': 33, 'be': 34, 'with': 35, 'this': 36, 'node': 37, 'list': 38, 'memory': 39, 'operating': 40, 'can': 41, 'called': 42, 'used': 43, 'one': 44, 'binary': 45, 'on': 46, 'between': 47, 'algorithms': 48, 'case': 49, 'from': 50, 'not': 51, 'set': 52, 'method': 53, 'when': 54, 'two': 55, 'each': 56, 'at': 57, 'kernel': 58, 'all': 59, 'problem': 60, 'nodes': 61, 'its': 62, 'number': 63, 'path': 64, 'program': 65, 'also': 66, 'order': 67, 'sorting': 68, 'address': 69, 'other': 70, 'bubble': 71, 'processes': 72, 'space': 73, 'any': 74, 'performance': 75, 'where': 76, 'linear': 77, 'if': 78, 'cpu': 79, 'state': 80

In [6]:
def split_dataset(x, y, percent = [0.85, 0.15] ):
	# number of examples
	data_len = len(x)
	lens = [ int(data_len*item) for item in percent ]

	trainX, trainY = x[:lens[0]], y[:lens[0]]
	testX, testY = x[lens[0]:lens[0]+lens[1]], y[lens[0]:lens[0]+lens[1]]
	# validX, validY = x[-lens[-1]:], y[-lens[-1]:]

	return (trainX,trainY), (testX,testY)

In [7]:
def intial_setup(path):
    metadata, idx_q, idx_a = load_data(path)
    (trainX, trainY), (testX, testY) = split_dataset(idx_q, idx_a)

    trainX = tl.prepro.remove_pad_sequences(trainX.tolist())
    trainY = tl.prepro.remove_pad_sequences(trainY.tolist())
    testX = tl.prepro.remove_pad_sequences(testX.tolist())
    testY = tl.prepro.remove_pad_sequences(testY.tolist())
    # validX = tl.prepro.remove_pad_sequences(validX.tolist())
    # validY = tl.prepro.remove_pad_sequences(validY.tolist())

    return (trainX, trainY), (testX, testY), metadata

In [17]:
(trainX, trainY), (testX, testY), metadata = intial_setup(path)
print(metadata)
src_vocab_size = len(metadata['idx2w'])
print(f'train x 0 >>>>>>>>>>>>>> {trainX[0]}')
emb_dim = 1024
word2index = metadata['w2idx']
index2word = metadata['idx2w']
batch_size = 32
num_steps = len(trainX)//batch_size
unk_id = word2index['unk']
pad_id = word2index['_']

start_id = src_vocab_size
end_id = src_vocab_size + 1

word2index.update({'start_id':start_id, 'end_id':end_id})
index2word.append('start_id')
index2word.append('end_id')
print(len(trainX))

src_vocab_size = tgt_vocab_size = src_vocab_size + 2

num_epochs = 40

vocab_size = src_vocab_size 

decoder_seq_length = 50
sentence = [index2word[w] for w in trainX[0]]
print(f'the index 2 word validity >>>>>>> {sentence}')

{'w2idx': {'_': 0, 'unk': 1, 'the': 2, 'a': 3, 'is': 4, 'of': 5, 'algorithm': 6, 'to': 7, 'in': 8, 'what': 9, 'you': 10, 'do': 11, 'and': 12, 'by': 13, 'that': 14, 'an': 15, 'for': 16, 'are': 17, 'how': 18, 'define': 19, 'mean': 20, 'explain': 21, 'it': 22, 'or': 23, 'process': 24, 'tree': 25, 'search': 26, 'system': 27, 'time': 28, 'sort': 29, 'as': 30, 'which': 31, 'graph': 32, 'data': 33, 'be': 34, 'with': 35, 'this': 36, 'node': 37, 'list': 38, 'memory': 39, 'operating': 40, 'can': 41, 'called': 42, 'used': 43, 'one': 44, 'binary': 45, 'on': 46, 'between': 47, 'algorithms': 48, 'case': 49, 'from': 50, 'not': 51, 'set': 52, 'method': 53, 'when': 54, 'two': 55, 'each': 56, 'at': 57, 'kernel': 58, 'all': 59, 'problem': 60, 'nodes': 61, 'its': 62, 'number': 63, 'path': 64, 'program': 65, 'also': 66, 'order': 67, 'sorting': 68, 'address': 69, 'other': 70, 'bubble': 71, 'processes': 72, 'space': 73, 'any': 74, 'performance': 75, 'where': 76, 'linear': 77, 'if': 78, 'cpu': 79, 'state': 80

In [18]:
model = Seq2seq(
        decoder_seq_length = decoder_seq_length,
        cell_enc=tf.keras.layers.GRUCell,
        cell_dec=tf.keras.layers.GRUCell,
        n_layer=3,
        n_units=256,
        embedding_layer=tl.layers.Embedding(vocabulary_size=vocab_size, embedding_size=emb_dim),
        )

[TL] Embedding embedding_2: (3113, 1024)
[TL] RNN rnn_7: cell: GRUCell, n_units: 256
[TL] RNN rnn_8: cell: GRUCell, n_units: 256
[TL] RNN rnn_9: cell: GRUCell, n_units: 256
[TL] RNN rnn_10: cell: GRUCell, n_units: 256
[TL] RNN rnn_11: cell: GRUCell, n_units: 256
[TL] RNN rnn_12: cell: GRUCell, n_units: 256
[TL] Reshape reshape_4
[TL] Dense  dense_2: 3113 No Activation
[TL] Reshape reshape_5
[TL] Reshape reshape_6


In [19]:
optimizer = tf.optimizers.Adam(learning_rate=0.001)

In [20]:
seeds = ["how do you explain algorithm",
          "what is a socket"]

In [21]:
trainX, trainY = shuffle(trainX, trainY, random_state=0)

In [22]:
for epoch in range(num_epochs):
    model.train()
    trainX, trainY = shuffle(trainX, trainY, random_state=0)
    total_loss, n_iter = 0, 0
    for x,y in tqdm(tl.iterate.minibatches(inputs=trainX,targets=trainY,batch_size=batch_size,shuffle=False),
                    total=num_steps,desc='Epoch[{}/{}]'.format(epoch + 1, num_epochs), leave=False):
        x =tl.prepro.pad_sequences(x)
        target_sequences = tl.prepro.sequences_add_end_id(y,end_id=end_id)
        target_sequences = tl.prepro.pad_sequences(target_sequences, maxlen= decoder_seq_length)
        decode_sequences = tl.prepro.sequences_add_start_id(y,start_id=start_id,remove_last=False)
        decode_sequences = tl.prepro.pad_sequences(decode_sequences, maxlen=decoder_seq_length)
        target_mask = tl.prepro.sequences_get_mask(target_sequences)
        with tf.GradientTape() as tape:
            ## compute outputs
            output = model(inputs = [x, decode_sequences])
                
            output = tf.reshape(output, [-1, vocab_size])
            ## compute loss and update model
            loss = tl.cost.cross_entropy_seq_with_mask(logits=output, target_seqs=target_sequences, input_mask=target_mask)

            grad = tape.gradient(loss, model.all_weights)
            optimizer.apply_gradients(zip(grad, model.all_weights))
            
            total_loss += loss
            n_iter += 1

        # printing average loss after every epoch
    print('Epoch [{}/{}]: loss {:.4f}'.format(epoch + 1, num_epochs, total_loss / n_iter))
    for seed in seeds:
        print("Query >", seed)
        top_n = 3
        for i in range(top_n):
            sentence = inference(seed, top_n)
            print(" >", ' '.join(sentence))



Epoch [1/40]: loss 6.3187
Query > how do you explain algorithm
 > the is is is the a a is a of
 > the is the the is the is is the a
 > the the is is a the the is of of a
Query > what is a socket
 > the the is a a the the the of the the
 > the the the is the the the the the a a the the a
 > the the the a is is a is of the


Epoch [2/40]: loss 5.9477
Query > how do you explain algorithm
 > a is is the is is a the of of the
 > a is the a a is of is of is of the of
 > a a a is of is is is of is the
Query > what is a socket
 > a is is the a is a is of
 > a is a the is of of of is
 > a a a is a is a is the


Epoch [3/40]: loss 5.8795
Query > how do you explain algorithm
 > a a the is is a is a the of of the the of of the of of of the a a the a a the a a a
 > a the is is the a is of the of the a the of the the a the the a a a
 > a the a a of is a a the the of a the a the of the a a a a
Query > what is a socket
 > a a the is a is is the the of of of of the a a of the a a a a a
 > a the the the a of of a a of the the of the a of the the a
 > a a the a a of the the a of of a of the of a the a


Epoch [4/40]: loss 5.7731
Query > how do you explain algorithm
 > a the is a the a algorithm a algorithm a a the of algorithm of of of a the a a a the the of of the of the the
 > a algorithm a algorithm algorithm the a a a the a a a a algorithm of of a a of of the of a of the the the the a the
 > a a a a the the algorithm a algorithm a algorithm of algorithm of algorithm of a the of a of the the the the of a a the a
Query > what is a socket
 > a algorithm is the algorithm algorithm a a a algorithm algorithm algorithm of a a a the a a the the a of the a of of of the a the of a a the
 > a the is the the the a the a algorithm algorithm of a algorithm algorithm of a a a the the a of a of a a a the of a
 > a the a a algorithm the algorithm of the a algorithm algorithm of the the the the a the of of of of a the the a of of a a the a a a the a the the a


Epoch [5/40]: loss 5.5164
Query > how do you explain algorithm
 > a a the a a algorithm for algorithm for the algorithm
 > a a the the algorithm is an a a a the a algorithm is the a algorithm is a the a algorithm
 > a the a algorithm for the a algorithm for a algorithm of the the the algorithm
Query > what is a socket
 > a algorithm is an a a sort is a the algorithm of a the the the a a the algorithm
 > a algorithm is the algorithm for algorithm is a algorithm is an the algorithm of the the a the the the algorithm
 > a a a algorithm of a algorithm of a algorithm


...

Epoch [36/40]: loss 0.1860
Query > how do you explain algorithm
 > is a cycle in function value by a array node that have empty the value of the heap usually adding they also they can share the value holds the address of all storage node that contains the root from one
 > is an optimal multiplication complexity of finding binary list in which out or small values to 2 or 3dimensional binary list
 > is a base of both every of gcd with their parent until right elements
Query > what is a socket
 > a graph in finding the minimum bounding box calculated subject to no constraints on that the orientation to any final state is graph the average search performance the time it is an integer combination
 > a graph is that it is a type of scheduling that can be used in an edge is called a simple sorting such with each other has a vertex in finding all weights v u u and so the second
 > a graph is that it takes a edge between the leaves x on an key than the root and the smallest node the prime from 

Epoch [37/40]: loss 0.1666
Query > how do you explain algorithm
 > is memory refers the data structure is likely that are the process to be determined to some addresses the machine address are user will be known as required the number of operations at one time in which
 > is an optimal outputsensitive technique of computing the convex common collection of points to n which is initial points in the directed sorting it is v
 > is a specialpurpose sorting algorithm which uses similar and insertion search that solves the graph that is connected as an optimization in a computer number and uses its values in the case and array to the shortest path x in
Query > what is a socket
 > a socket provides a connection between one of two applications each abstract basic terminal nodes node in the certain array
 > a simple diagram where n can have any obstacle
 > a graph which has only two vertex can be called order can not have one or more than we pointers can be root with each other v nodes exactly 

Epoch [38/40]: loss 0.1500
Query > how do you explain algorithm
 > is a cycle in value with the input node called adjacent empty this algorithm are distinct to an order to the goal subtree in preceding complex the root
 > is an approach of an infinite function or that strings with their pair or more numbers in found with each vertex in first they is v
 > is an item to an unsorted sequence but with all edge applications want on to any input array or numerical level or an abacus
Query > what is a socket
 > a socket is used to make connection between one of two subproblems are more than we applications can be several find a amount of time they has any parent then the linked list pages are another node in the list
 > a thread is that processes which are not yet mechanisms wait that has only requires they does requires this time they are not resources it is not available as known whenever the thread structure it provides the user that they
 > a thread that takes a source have one type betwe

Epoch [39/40]: loss 0.1389
Query > how do you explain algorithm
 > is a specialpurpose diagram in by least the elements in which each node are called as least out or less than
 > is an approach and computing the function that is not sequences and conquer as directed as every vertex is a distance point in the linked list but it is connected as an edge called a numerical specified
 > is a specialpurpose diagram
Query > what is a socket
 > a recursive diagram with be classified as their levenshtein node that have a linked list has only adjacent adjacent occurs it is applicable which method which is either integer
 > a recursive algorithm for determining roots polynomials in many sound when pronounced in english
 > a lineartime algorithm used for indexing of words by their pronunciation may be found


Epoch [40/40]: loss 0.1235
Query > how do you explain algorithm
 > is a specialpurpose algorithm meaning on the shortest paths between data
 > is memory letter when one data is more useful in list such algorithm such as implementation or find the cycle in linked lists occurs
 > is a generic means in an undirected graph with every two more more adjacent point will not have the same combination ie as it occupies ie than the goal child node is composed and a table organized so the second
Query > what is a socket
 > a lineartime online algorithm to compute a multiplication algorithm or building graph for some integer vertex it uses depthfirst as a integer integer integer order called the travelling level using a integer whose
 > a thread is a basic unit and part points of an array node on an event of any at the first the element has this is called underflow
 > a socket is usually as an extension of processes it is execution and particular programs the most time units


In [14]:
def inference(seed, top_n):
        model.eval()
        seed_id = [word2index.get(w, unk_id) for w in word_tokenize(seed)]
        sentence_id = model(inputs=[[seed_id]], seq_length=40, start_token=start_id, top_n = top_n)
        sentence = []
        for w_id in sentence_id[0]:
            w = index2word[w_id]
            if w == 'end_id':
                break
            sentence = sentence + [w]
        return sentence

In [48]:
test = "what is your name"

In [49]:
for word in inference(test,1):
    print(word, end=' ')

sort the first and second half of the list separately then merge the sorted lists 

In [52]:
model.save_weights('my_model')

[TL] [*] Saving TL weights into my_model
[TL] [*] Saved
